In [1]:
import os
import cv2
import torch
from ultralytics import YOLO
import pandas as pd
import joblib
import numpy as np

# ==============================================================================
# BAGIAN I: SETUP DAN PEMUATAN SEMUA MODEL & DATA
# ==============================================================================
print("--- Memuat semua model dan data yang diperlukan ---")

# --- 1. Muat Model YOLO ---
try:
    yolo_model = YOLO("model_26-05_1.pt")
    print("Model YOLO berhasil dimuat.")
except Exception as e:
    print(f"Error memuat model YOLO: {e}")
    exit()

# --- 2. Muat Faktor Konversi gram_per_pixel ---
try:
    conversion_factors_df = pd.read_csv('pixel_dataset.csv')
    gram_per_pixel_map = pd.Series(conversion_factors_df.gram_per_pixel.values,
                                   index=conversion_factors_df.food_type).to_dict()
    print("Data faktor konversi gram_per_pixel berhasil dimuat.")
except FileNotFoundError:
    print("Error: File 'pixel_dataset.csv' tidak ditemukan. Estimasi berat tidak dapat dilakukan.")
    gram_per_pixel_map = {}
except Exception as e:
    print(f"Error saat memuat 'pixel_dataset.csv': {e}")
    gram_per_pixel_map = {}

# --- 3. Muat Model SVR dan Komponennya ---
try:
    svr_nutrition_model = joblib.load('svr_multi_output_model.pkl')
    scaler = joblib.load('svr_scaler.pkl')
    feature_names = joblib.load('svr_feature_names.pkl')
    print("Model SVR untuk estimasi nutrisi berhasil dimuat.")
except FileNotFoundError:
    print("Error: Salah satu file model SVR (.pkl) tidak ditemukan. Estimasi nutrisi tidak dapat dilakukan.")
    svr_nutrition_model = None # Set ke None jika tidak ada
except Exception as e:
    print(f"Error saat memuat model SVR: {e}")
    svr_nutrition_model = None


# ==============================================================================
# BAGIAN II: PROSES GAMBAR DAN ESTIMASI BERTINGKAT
# ==============================================================================

# --- Tentukan file input ---
file_name = "ayam-110gr_nasi-110gr_1.jpg"
input_path = f"run-tests/{file_name}"
print(f"\n--- Memproses gambar: {file_name} ---")

# --- Pra-pemrosesan Gambar ---
image = cv2.imread(input_path)
if image is None:
    print(f"Error: Tidak dapat membaca gambar dari path: {input_path}")
    exit()

target_size = (640, 640)
resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

# --- 4. Jalankan Deteksi dan Segmentasi YOLO ---
results = yolo_model(source=resized_image, save=False, verbose=False) # verbose=False agar output tidak terlalu ramai
print("Deteksi dan segmentasi YOLO selesai.")

# Simpan dan tampilkan hasil segmentasi
base_name, ext = os.path.splitext(file_name)
output_path = f"run-tests/{base_name}_predict{ext}"
annotated_frame = results[0].plot()
cv2.imwrite(output_path, annotated_frame)
# results[0].show() # Opsional, hapus komentar jika ingin menampilkan jendela gambar

# --- 5. Tahap 1: Hitung Estimasi Berat dari Piksel ---
detected_food_pixels = {}
if results[0].masks is not None and hasattr(results[0].boxes, 'cls'):
    for i, mask in enumerate(results[0].masks.data):
        area = torch.sum(mask).item()
        class_id = int(results[0].boxes.cls[i].item())
        class_name = results[0].names[class_id]
        
        # Akumulasi piksel per jenis makanan
        detected_food_pixels[class_name] = detected_food_pixels.get(class_name, 0) + area

estimated_weights = {}
if gram_per_pixel_map:
    print("\n--- Hasil Tahap 1: Estimasi Berat Makanan ---")
    for food_name, total_pixels in detected_food_pixels.items():
        if food_name in gram_per_pixel_map:
            g_per_pixel = gram_per_pixel_map[food_name]
            estimated_weight = total_pixels * g_per_pixel
            estimated_weights[food_name] = estimated_weight
            print(f"-> {food_name}: Total Pixels = {total_pixels:.0f}, Estimasi Berat = {estimated_weight:.2f} gram")
        else:
            print(f"-> {food_name}: Total Pixels = {total_pixels:.0f}, Faktor gram/piksel tidak ditemukan.")
else:
    print("Tidak dapat melakukan estimasi berat karena data faktor konversi tidak dimuat.")

# --- 6. Tahap 2: Hitung Estimasi Nutrisi dari Estimasi Berat ---
if estimated_weights and svr_nutrition_model is not None:
    print("\n--- Hasil Tahap 2: Estimasi Nutrisi dari Estimasi Berat ---")
    
    total_nutrition = {'Carbohydrates': 0, 'Protein': 0, 'Fat': 0, 'Calories': 0}
    
    for food_name, weight in estimated_weights.items():
        # Siapkan data untuk model SVR
        new_data = pd.DataFrame([{'food_type': food_name, 'portion_size': weight}])
        new_data_encoded = pd.get_dummies(new_data)
        new_data_reindexed = new_data_encoded.reindex(columns=feature_names, fill_value=0)
        new_data_scaled = scaler.transform(new_data_reindexed)
        
        # Lakukan prediksi nutrisi
        prediction = svr_nutrition_model.predict(new_data_scaled)
        
        # Ubah nilai negatif menjadi 0
        adjusted_prediction = np.maximum(0, prediction[0])
        
        # Tampilkan hasil untuk item ini
        print(f"\nEstimasi Nutrisi untuk '{food_name}' (berat {weight:.2f} g):")
        print(f"  - Karbohidrat: {adjusted_prediction[0]:.2f} g")
        print(f"  - Protein:       {adjusted_prediction[1]:.2f} g")
        print(f"  - Lemak:         {adjusted_prediction[2]:.2f} g")
        print(f"  - Kalori:        {adjusted_prediction[3]:.0f} kkal")
        
        # Akumulasi total nutrisi
        total_nutrition['Carbohydrates'] += adjusted_prediction[0]
        total_nutrition['Protein'] += adjusted_prediction[1]
        total_nutrition['Fat'] += adjusted_prediction[2]
        total_nutrition['Calories'] += adjusted_prediction[3]
        
    # Tampilkan total estimasi nutrisi untuk seluruh makanan di gambar
    print("\n--- Total Estimasi Nutrisi di Gambar ---")
    print(f"  - Total Karbohidrat: {total_nutrition['Carbohydrates']:.2f} g")
    print(f"  - Total Protein:       {total_nutrition['Protein']:.2f} g")
    print(f"  - Total Lemak:         {total_nutrition['Fat']:.2f} g")
    print(f"  - Total Kalori:        {total_nutrition['Calories']:.0f} kkal")
    
elif not detected_food_pixels:
     print("\nTidak ada makanan yang terdeteksi, estimasi nutrisi tidak dilakukan.")
else:
    print("\nTidak dapat melakukan estimasi nutrisi karena model SVR tidak dimuat.")

--- Memuat semua model dan data yang diperlukan ---
Model YOLO berhasil dimuat.
Data faktor konversi gram_per_pixel berhasil dimuat.
Model SVR untuk estimasi nutrisi berhasil dimuat.

--- Memproses gambar: ayam-110gr_nasi-110gr_1.jpg ---
Deteksi dan segmentasi YOLO selesai.

--- Hasil Tahap 1: Estimasi Berat Makanan ---
-> Nasi Putih: Total Pixels = 101276, Estimasi Berat = 109.38 gram
-> Ayam Goreng - Dada: Total Pixels = 59708, Estimasi Berat = 116.43 gram

--- Hasil Tahap 2: Estimasi Nutrisi dari Estimasi Berat ---

Estimasi Nutrisi untuk 'Nasi Putih' (berat 109.38 g):
  - Karbohidrat: 28.52 g
  - Protein:       3.12 g
  - Lemak:         0.35 g
  - Kalori:        129 kkal

Estimasi Nutrisi untuk 'Ayam Goreng - Dada' (berat 116.43 g):
  - Karbohidrat: 2.22 g
  - Protein:       33.22 g
  - Lemak:         19.03 g
  - Kalori:        310 kkal

--- Total Estimasi Nutrisi di Gambar ---
  - Total Karbohidrat: 30.75 g
  - Total Protein:       36.34 g
  - Total Lemak:         19.39 g
  - Tota